In [2]:
import pandas as pd
import re
from io import StringIO
from dateutil import parser as date_parser
import uuid

In [6]:
FILE_PATH = '/Users/ransela/Desktop/data_science_degree/4th_year/spring/Data Analysis and Visualization Lab/project/Whatsapp_webApp_-Django-/RAG/RAG_data/raw_data/translated_Maayan.txt'

USER_MAP = {
    'Maayan Sade': 'u_maayan',
    'Barbara': 'u_barbara',
}

In [7]:
def parse_chat_log(file_path):
    """
    Reads a chat log file and robustly parses each message line into structured data.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            chat_content = f.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure the file is present.")
        return pd.DataFrame()

    pattern = re.compile(
        r'^(?:.*?)\s*\[(\d{1,2}/\d{1,2}/\d{4}),\s*(\d{1,2}:\d{2}:\d{2})\s*(?:[AP]M)?\]\s*(.+?):\s*(.*)',
        re.IGNORECASE | re.MULTILINE
    )

    data = []

    for line in StringIO(chat_content):
        line = line.strip()
        if not line:
            continue

        match = pattern.match(line)

        if match:
            date_str, time_str, speaker, message = match.groups()

            clean_speaker = re.sub(r'\[.*?\]|\(|\)|\u200e|\u200f', '', speaker).strip()

            data.append({
                'Date': date_str,
                'Time': time_str,
                'Speaker': clean_speaker,
                'Message': message.strip()
            })
        elif data:
            data[-1]['Message'] += ' ' + line
    return pd.DataFrame(data)

def transform_to_sql_schema(df):
    """
    Transforms the raw DataFrame into the required 'messages' table schema,
    including the generated 'conv_id'.
    """
    if df.empty:
        return pd.DataFrame()

    unique_speakers = df['Speaker'].unique()
    df['sender_user_id'] = df['Speaker'].map(USER_MAP)
    receiver_id_map = {
        USER_MAP['Maayan Sade']: USER_MAP['Barbara'],
        USER_MAP['Barbara']: USER_MAP['Maayan Sade']
    }
    df['receiver_user_id'] = df['sender_user_id'].map(receiver_id_map)

    def generate_id_and_timestamp(row):
        msg_id = str(uuid.uuid4())
        try:
            dt_obj = date_parser.parse(f"{row['Date']} {row['Time']}", dayfirst=True)
            sent_at = dt_obj.strftime('%Y-%m-%d %H:%M:%S')
        except Exception:
            dt_obj = pd.NaT
            sent_at = None
        return pd.Series([msg_id, sent_at, dt_obj])

    df[['msg_id', 'sent_at','dt_object']] = df.apply(generate_id_and_timestamp, axis=1)
    user_ids = sorted(USER_MAP.values())
    df['conv_id'] = 'chat:' + '_'.join(user_ids)
    df = df.sort_values(by='dt_object').reset_index(drop=True)
    df = df.rename(columns={'Message': 'text'})
    df = df.sort_values(['conv_id', 'sent_at']).reset_index(drop=True)

    block_id = (
        (df['conv_id'] != df['conv_id'].shift()) |
        (df['sender_user_id'] != df['sender_user_id'].shift())
    ).cumsum()

    df['block_id'] = block_id
    df_merged = (
        df.groupby('block_id', as_index=False)
          .agg({
              'msg_id': 'first',
              'conv_id': 'first',
              'sender_user_id': 'first',
              'receiver_user_id': 'first',
              'sent_at': 'min',
              'text': lambda x: "\n".join(x)
          })
    )
    df_merged = df_merged.drop(columns=['block_id'])
    df_merged = df_merged.sort_values(by='sent_at').reset_index(drop=True)
    df_merged['conv_turn'] = (df_merged['sender_user_id'] == df_merged['receiver_user_id'].shift()).cumsum() +1
    df_merged['next_text'] = df_merged['text'].shift(-1)
    df_merged['next_sender'] = df_merged['sender_user_id'].shift(-1)
    df_merged['answer'] = df_merged['next_text']
    df_merged = df_merged.drop(columns=['next_text', 'next_sender'])
    df_merged['answer'] = df_merged['answer'].fillna('<EOC>') #END OF CONVERSATION


    final_columns = [
        'msg_id',
        'conv_id',
        'conv_turn',
        'sender_user_id',
        'receiver_user_id',
        'sent_at',
        'text',
        'answer'
    ]

    final_df = df_merged[final_columns]
    return final_df

In [8]:
raw_df = parse_chat_log(FILE_PATH)
df_messages = transform_to_sql_schema(raw_df)

In [10]:
df_messages.to_csv('/Users/ransela/Desktop/data_science_degree/4th_year/spring/Data Analysis and Visualization Lab/project/Whatsapp_webApp_-Django-/RAG/RAG_data/KB_data.csv',index=False)
